#### Predictions from the files on the bronze zone



Three steps in this notebook to achieving our objective:
- open the zarr from Azure, and using it inside the notebook
- making the predictions (for different thresholds) with the datasets from the bronze zone
- saving the file only with the indexes and the predictions

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
from data_integration_utils.azure_blob_manager.omic_blob_manager import OmicBronzeBSM, OmicGoldBSM
import zarr
from dotenv import load_dotenv
import json

load_dotenv(dotenv_path=".env")

from tqdm.auto import tqdm

from utils import (
    execute_cancer_finder,
    execute_sctab,
    unified_models_output
)

pd.options.mode.chained_assignment = None 

/Users/jhonatan/Desktop/Verifying_metrics/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jhonatan/Desktop/Verifying_metrics/.venv/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [16]:
# Code to setup the doppler, I used a file token_doppler with my token to use it

from token_doppler import TOKEN
from dopplersdk import DopplerSDK
import os

doppler = DopplerSDK()
doppler.set_access_token(TOKEN)
results = doppler.projects.list()

project = "data-integration-omic"
config =  "dev"
secrets = doppler.secrets.list(config, project)

def charge_variable_doppler(variable):
    os.environ[variable] = secrets.secrets[variable]['computed']

charge_variable_doppler("AZURE_DATASETS_ACCOUNT_URL")
charge_variable_doppler("SLACK_CONF")
charge_variable_doppler('AZURE_DATASETS_ACCOUNT_URL')
charge_variable_doppler('DATA_MLFLOW_SOURCE_CONTAINER_NAME')
charge_variable_doppler('DATA_MLFLOW_SOURCE_ACCOUNT_URL')

## Is not the dataintdevs
os.environ["AZURE_DATASETS_ACCOUNT_URL"] = 'https://dataintepigene.blob.core.windows.net/'

In [59]:
datasets_sctab = [
    "GSE132257",
    "GSE132465",
    "GSE178341",
    "GSE159929",
    "GSE200996",
    "GSE139555",
    "GSE116256",
    "GSE234129",
    "GSE159115",
    "GSE180298",
    "BerlinSC",
    "SCP1288",
    "Tabula_sapiens_Bladder",
    "Tabula_sapiens_Blood",
    "Tabula_sapiens_Bone_Marrow",
    "Tabula_sapiens_Eye",
    "Tabula_sapiens_Fat",
    "Tabula_sapiens_Heart",
    "Tabula_sapiens_Kidney",
    "Tabula_sapiens_Liver",
    "Tabula_sapiens_Large_Intestine",
    "Tabula_sapiens_Lung",
    "Tabula_sapiens_Lymph_Node",
    "Tabula_sapiens_Mammary",
    "Tabula_sapiens_Muscle",
    "Tabula_sapiens_Pancreas",
    "Tabula_sapiens_Prostate",
    "Tabula_sapiens_Skin",
    "Tabula_sapiens_Small_Intestine",
    "Tabula_sapiens_Spleen",
    "Tabula_sapiens_Thymus",
    "Tabula_sapiens_Tongue",
    "Tabula_sapiens_Trachea",
    "Tabula_sapiens_Uterus",
    "Tabula_sapiens_Vasculature"
]

slack_conf = json.loads(os.environ["SLACK_CONF"])

In [8]:
def load_adata_bronze(dataset):
    bronze_bsm = OmicBronzeBSM()
    bronze_store = zarr.open_group(zarr.ABSStore(client=bronze_bsm.container_client, prefix=f"bronze/scrnaseq/{dataset}_raw_data.zarr"))
    adata = anndata.read_zarr(bronze_store)
    return adata

In [ ]:
#dataset = datasets_sctab[0]
#adata = load_adata_bronze(dataset)

In [41]:
adata.obs.head()

,donor_id,sample_type,Status,Sample,paper_cell_type_1,dataset,sample_id,epigene_cell_type,epigene_cell_subtype
GSM3855020_AAACCTGAGATCGGGT,SMC13,Normal,Frozen,SMC13N-A1-F,T cells,GSE132257,GSM3855020,T cell,"T cell, NOS"
GSM3855020_AAACCTGAGGATTCGG,SMC13,Normal,Frozen,SMC13N-A1-F,T cells,GSE132257,GSM3855020,T cell,"T cell, NOS"
GSM3855020_AAACCTGGTACTCAAC,SMC13,Normal,Frozen,SMC13N-A1-F,T cells,GSE132257,GSM3855020,T cell,"T cell, NOS"
GSM3855020_AAACGGGAGCTTTGGT,SMC13,Normal,Frozen,SMC13N-A1-F,B cells,GSE132257,GSM3855020,B cell,"B cell, NOS"
GSM3855020_AAACGGGCATCCAACA,SMC13,Normal,Frozen,SMC13N-A1-F,B cells,GSE132257,GSM3855020,B cell,"B cell, NOS"


In [44]:
adata.var.head()

""
RP11-34P13.3
FAM138A
OR4F5
RP11-34P13.7
RP11-34P13.8


In [9]:
def predict_dataset(adata, dataset, sctab, cancerfinder, seed = None ):
    
    list_intersect_cf = None
    list_intersect_sctab = None

    if seed is None:
        if cancerfinder:
            version = 'CF'
            adata, intersec_cf, list_intersect_cf = execute_cancer_finder(adata,slack_conf)
            intersec_cf = round(intersec_cf, 2)
            path = f"./metrics_annotation_models/predictions/{dataset}_{version}_{intersec_cf}.csv"
        if sctab:
            version = 'scTab'
            adata, intersec_sc, list_intersect_sctab = execute_sctab(adata)
            intersec_sc = round(intersec_sc, 2)
            path = f"./metrics_annotation_models/predictions/{dataset}_{version}_{intersec_sc}.csv"
        if sctab or cancerfinder:
            adata = unified_models_output(adata)
        if sctab and cancerfinder:
            version = 'complete'
            path = f"./metrics_annotation_models/predictions/{dataset}_{version}_{intersec_sc}_{intersec_cf}.csv"
    else:
        path = path.split(".csv")[0] + f'_{str(seed)}' + path.split(".csv")[1]
    return adata, path, list_intersect_cf, list_intersect_sctab

def save_reduced_observations(adata, path, sctab = True, cancerfinder = True):
    columns = ["epigene_cell_type", 
            "epigene_cell_subtype", 
            "metapipeline_automatic_cell_type"
            ]
    if cancerfinder:
        columns.append("prediction_CancerFinder")
    if sctab:
        columns.append("predicted_mapped_scTab")
    adata.obs[columns].to_csv(path)


In [ ]:
# Test
dataset = datasets_sctab[0]

adata = load_adata_bronze(dataset)
adata_final, path, _, _ = predict_dataset(adata, dataset, cancerfinder= True, sctab= True)
save_reduced_observations(adata, path, sctab = True, cancerfinder= True)

In [47]:
### Now the way to follow as a pipeline is to use the functions in the following way:
### load_adata_bronze -> predict_and_save_dataset

### now the idea is to use the load_adata_bronze to get the adata, after filter using 
### another function specifying the version with the number of the seed, and after 
### that make the predictions ans save passing the function predict_and_save_dataset.

## Bootstrap for genes dropout

In [55]:
adata.n_vars

33694

In [10]:
n_genes_sctab = 19331 
n_genes_cancerfinder = 0

# def dropout_genes(adata, drop_fraction = 0.1, random_state = 42):
#     """
#     Function to make a gene dropout on the dataset oy choose.

#     adata: 
#     drop_fraction: fraction of the dataset you choose to drop.
#     random_state: chose the seed for the random method to assure reproducibility
#     """
#     n_genes = adata.n_vars
#     n_drop = int(n_genes*drop_fraction)

#     np.random.seed(random_state)
#     drop_indices = np.random.choice(n_genes, n_drop, replace= False)

#     keep_mask = np.ones(n_genes, dtype = bool)
#     keep_mask[drop_indices] = False

#     adata_dropped = adata[:, keep_mask].copy()

#     return adata_dropped


def dropout_genes(adata, gene_list, n_genes_model ,drop_fraction=0.1, random_state=42):
    """
    Filters the AnnData object to the provided list of genes, then drops a fraction of them at random.

    Parameters:
    -----------
    adata : AnnData
        The input annotated data matrix.
    gene_list : list of str
        The list of genes (variables) to retain before dropout.
    n_genes_model: int
        Quantity of genes for the models we are using in the moment.
    drop_fraction : float, optional (default=0.1)
        Fraction of the filtered genes to randomly drop.
    random_state : int, optional (default=42)
        Seed for reproducibility.

    Returns:
    --------
    AnnData
        A new AnnData object with the selected genes filtered and a subset dropped.
    """
    # Filter adata for selected genes
    adata_filtered = adata[:, gene_list].copy()
    
    # Dropout calculation
    n_genes = n_genes_model
    n_drop = int(n_genes * drop_fraction)
    if n_drop == 0:
        return adata_filtered  # Return unchanged if nothing is to be dropped

    np.random.seed(random_state)
    drop_indices = np.random.choice(n_genes, n_drop, replace=False)

    keep_mask = np.ones(n_genes, dtype=bool)
    keep_mask[drop_indices] = False

    adata_dropped = adata_filtered[:, keep_mask].copy()
    
    return adata_dropped


### Predictions made on the different datasets from the list for scTab

In [11]:
datasets_sctab = [
    "GSE132257",
    "GSE132465",
    "GSE178341",
    "GSE159929",
    "GSE200996",
    "GSE139555",
    "GSE116256",
    "GSE234129",
    "GSE159115",
    "GSE180298",
    "BerlinSC",
    "SCP1288",
    "Tabula_sapiens_Bladder",
    "Tabula_sapiens_Blood",
    "Tabula_sapiens_Bone_Marrow",
    "Tabula_sapiens_Eye",
    "Tabula_sapiens_Fat",
    "Tabula_sapiens_Heart",
    "Tabula_sapiens_Kidney",
    "Tabula_sapiens_Liver",
    "Tabula_sapiens_Large_Intestine",
    "Tabula_sapiens_Lung",
    "Tabula_sapiens_Lymph_Node",
    "Tabula_sapiens_Mammary",
    "Tabula_sapiens_Muscle",
    "Tabula_sapiens_Pancreas",
    "Tabula_sapiens_Prostate",
    "Tabula_sapiens_Skin",
    "Tabula_sapiens_Small_Intestine",
    "Tabula_sapiens_Spleen",
    "Tabula_sapiens_Thymus",
    "Tabula_sapiens_Tongue",
    "Tabula_sapiens_Trachea",
    "Tabula_sapiens_Uterus",
    "Tabula_sapiens_Vasculature"
]

datasets_cancerfinder = [
    "BerlinSC", 
    "GSE132465", 
    "SCP1288", 
    "GSE159115"
]

In [17]:
slack_conf = json.loads(os.environ["SLACK_CONF"])

non_successful_datasets = []
cf = True
sct = True
for dataset in datasets_sctab:
    try:
        adata = load_adata_bronze(dataset)
        adata, path, _, _ = predict_dataset(adata, dataset, cancerfinder= cf, sctab= sct)
        save_reduced_observations(adata, path, sctab = sct, cancerfinder= cf)
    except Exception as e:
        print(f"Failed to load {dataset}: {e}")
        non_successful_datasets.append(dataset)
        continue  # skip to the next dataset

/var/folders/x_/7kq86smn27b7j7l6xzvtjlm00000gn/T/ipykernel_71111/3344461464.py:6: FutureWarning: The ABSStore is deprecated and will be removed in a Zarr-Python version 3, see https://github.com/zarr-developers/zarr-python/issues/1274 for more information.
  adata = load_adata_bronze(dataset)


Genes in intersection: 17499
Adding 6792 missing genes with zero values.
Matrix shape: (24291, 18409)
AnnData object with n_obs × n_vars = 24291 × 18409
/n
AnnData object with n_obs × n_vars = 24291 × 18409
begin 0
begin 1


/Users/jhonatan/Desktop/Verifying_metrics/mlflow-models/packages/SequencingCancerFinder/CancerFinder.py:224: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  predict_df['predict'][predict_df['predict'] > self.args.threshold] = 1
/Users/jhonatan

            feature_id feature_name
0      ENSG00000186092        OR4F5
1      ENSG00000284733       OR4F29
2      ENSG00000284662       OR4F16
3      ENSG00000187634       SAMD11
4      ENSG00000188976        NOC2L
...                ...          ...
19326  ENSG00000288702       UGT1A3
19327  ENSG00000288705       UGT1A5
19328  ENSG00000182484       WASH6P
19329  ENSG00000288622   PDCD6-AHRR
19330  ENSG00000285815  GET1-SH3BGR

[19331 rows x 2 columns]
##################################################################
The quantity of genes in this dataset is 33694
The quantity of genes in the model is 19331
The percentage of intesection for sctab is 0.9458382908282034
##################################################################


100%|██████████| 9/9 [00:10<00:00,  1.12s/it]
/Users/jhonatan/Desktop/Verifying_metrics/utils.py:1054: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_combined.obs[columns_scTab] = df_processed[columns_processed]
/var/folders/x_/7kq86smn27b7j7l6xzvtjlm00000gn/T/ipykernel_71111/3344461464.py:6: FutureWarning: The ABSStore is deprecated and will be removed in a Zarr-Python version 3, see https://github.com/zarr-developers/zarr-python/issues/1274 for more information.
  adata = load_adata_bronze(dataset)


...

/Users/jhonatan/Desktop/Verifying_metrics/mlflow-models/packages/SequencingCancerFinder/CancerFinder.py:224: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  predict_df['predict'][predict_df['predict'] > self.args.threshold] = 1
/Users/jhonatan

            feature_id feature_name
0      ENSG00000186092        OR4F5
1      ENSG00000284733       OR4F29
2      ENSG00000284662       OR4F16
3      ENSG00000187634       SAMD11
4      ENSG00000188976        NOC2L
...                ...          ...
19326  ENSG00000288702       UGT1A3
19327  ENSG00000288705       UGT1A5
19328  ENSG00000182484       WASH6P
19329  ENSG00000288622   PDCD6-AHRR
19330  ENSG00000285815  GET1-SH3BGR

[19331 rows x 2 columns]
##################################################################
The quantity of genes in this dataset is 33694
The quantity of genes in the model is 19331
The percentage of intesection for sctab is 0.9458382908282034
##################################################################


100%|██████████| 32/32 [00:32<00:00,  1.01s/it]
/Users/jhonatan/Desktop/Verifying_metrics/utils.py:1054: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_combined.obs[columns_scTab] = df_processed[columns_processed]
/var/folders/x_/7kq86smn27b7j7l6xzvtjlm00000gn/T/ipykernel_71111/3344461464.py:6: FutureWarning: The ABSStore is deprecated and will be removed in a Zarr-Python version 3, see https://github.com/zarr-developers/zarr-python/issues/1274 for more information.
  adata = load_adata_bronze(dataset)


Genes in intersection: 16833
Adding 7458 missing genes with zero values.
Matrix shape: (24291, 370115)
AnnData object with n_obs × n_vars = 24291 × 370115
/n
AnnData object with n_obs × n_vars = 24291 × 370115
begin 0
begin 1
begin 2
begin 3
begin 4
begin 5
begin 6
begin 7
begin 8
begin 9
begin 10
begin 11
begin 12
begin 13
begin 14
begin 15
begin 16
begin 17
begin 18
begin 19


: 

#### Calculating the predictions for different quantities of intersections:

- Have to verify which are the number of intersection for the datasets because I can calculate only for a smaller number for them
- 